# Análise Exploratória Orientada à Construção de Features

Este notebook tem como objetivo realizar uma análise exploratória simples e direcionada dos atributos que darão origem às features utilizadas no treinamento do modelo de regressão.

O foco do modelo é a previsão do valor do frete de pedidos, e, por esse motivo, a análise aqui apresentada não é genérica, mas concentrada exclusivamente nas variáveis consideradas relevantes para esse objetivo.

Serão avaliados aspectos básicos de qualidade dos dados: valores nulos, inconsistências e plausibilidade dos valores.

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
BASE_PATH = Path("../../../")
DATA_RAW = BASE_PATH/"data"/"raw"

order_items = pd.read_csv(DATA_RAW / "olist_order_items_dataset.csv")
products = pd.read_csv(DATA_RAW / "olist_products_dataset.csv")
orders = pd.read_csv(DATA_RAW / "olist_orders_dataset.csv")
customers = pd.read_csv(DATA_RAW / "olist_customers_dataset.csv")
sellers = pd.read_csv(DATA_RAW / "olist_sellers_dataset.csv")
geolocation = pd.read_csv(DATA_RAW / "olist_geolocation_dataset.csv")
payments = pd.read_csv(DATA_RAW / "olist_order_payments_dataset.csv")

## Status do Pedido (`order_status`)

Foi avaliada a proporção de pedidos com status de entrega concluída.  
Para o treinamento do modelo, serão considerados apenas pedidos com status de entrega concluída (`delivered`), por representarem transações efetivamente finalizadas.

In [3]:
total_pedidos = len(orders)
pedidos_entregues = orders[orders['order_status'] == 'delivered'].shape[0]

print(f"Total de pedidos: {total_pedidos}")
print(f"Pedidos entregues: {pedidos_entregues} ({pedidos_entregues/total_pedidos:.2%})")

Total de pedidos: 99441
Pedidos entregues: 96478 (97.02%)


## Valor do Frete (`freight_value`)

A presença de valores iguais a zero é baixa (0,34%) e pode ser explicada por políticas de frete grátis, não sendo considerada uma inconsistência.

Não foram observadas inconsistências relevantes ou valores extremos nas suas distribuições.

In [4]:
target = order_items['freight_value']

nulos = target.isnull().sum()
zeros = (target == 0).sum()
negativos = (target < 0).sum()

print(f"Valores Nulos: {nulos} ({nulos/len(target):.2%})")
print(f"Valores Zero: {zeros} ({zeros/len(target):.2%})")
print(f"Valores Negativos: {negativos}")

display(target.describe())

Valores Nulos: 0 (0.00%)
Valores Zero: 383 (0.34%)
Valores Negativos: 0


count    112650.000000
mean         19.990320
std          15.806405
min           0.000000
25%          13.080000
50%          16.260000
75%          21.150000
max         409.680000
Name: freight_value, dtype: float64

## Dimensões e Peso dos Produtos (`product_weight_g`, `product_length_cm`, `product_height_cm`, `product_width_cm`)

Foram identificados alguns registros com valores nulos e valores iguais a zero no peso do produto. Esses casos não são fisicamente plausíveis.

Esses registros foram **apenas identificados nesta etapa** e **serão tratados posteriormente** na etapa de limpeza dos dados.

Não foram observadas inconsistências relevantes ou valores extremos nas suas distribuições.

In [5]:
dim_cols = [
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
]

dim = products[dim_cols]
n = len(dim)

nulos = dim.isna().sum().to_frame("qtd_nulos")
nulos["pct_nulos"] = nulos["qtd_nulos"] / n * 100
display(nulos)

zeros = (dim == 0).sum().to_frame("qtd_zeros")
zeros["pct_zeros"] = zeros["qtd_zeros"] / n * 100
display(zeros)

negativos = (dim < 0).sum().to_frame("qtd_negativos")
negativos["pct_negativos"] = negativos["qtd_negativos"] / n * 100
display(negativos)

display(dim.describe())

,qtd_nulos,pct_nulos
product_weight_g,2,0.00607
product_length_cm,2,0.00607
product_height_cm,2,0.00607
product_width_cm,2,0.00607


,qtd_zeros,pct_zeros
product_weight_g,4,0.012139
product_length_cm,0,0.000000
product_height_cm,0,0.000000
product_width_cm,0,0.000000


,qtd_negativos,pct_negativos
product_weight_g,0,0.0
product_length_cm,0,0.0
product_height_cm,0,0.0
product_width_cm,0,0.0


,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32949.000000,32949.000000,32949.000000,32949.000000
mean,2276.472488,30.815078,16.937661,23.196728
std,4282.038731,16.914458,13.637554,12.079047
min,0.000000,7.000000,2.000000,6.000000
25%,300.000000,18.000000,8.000000,15.000000
50%,700.000000,25.000000,13.000000,20.000000
75%,1900.000000,38.000000,21.000000,30.000000
max,40425.000000,105.000000,105.000000,118.000000


## Preço dos Itens (`price`)

Não foram identificados valores nulos, iguais a zero ou negativos.

Não foram observadas inconsistências relevantes ou valores extremos nas suas distribuições.

In [6]:
target = order_items["price"] 
n = len(target) 

nulos = target.isna().sum() 
zeros = (target == 0).sum() 
negativos = (target < 0).sum() 

print(f"Valores Nulos: {nulos} ({nulos/n:.2%})") 
print(f"Valores Zero: {zeros} ({zeros/n:.2%})") 
print(f"Valores Negativos: {negativos} ({negativos/n:.2%})") 

display(target.describe())

Valores Nulos: 0 (0.00%)
Valores Zero: 0 (0.00%)
Valores Negativos: 0 (0.00%)


count    112650.000000
mean        120.653739
std         183.633928
min           0.850000
25%          39.900000
50%          74.990000
75%         134.900000
max        6735.000000
Name: price, dtype: float64

## Estados do Cliente e do Vendedor (`customer_state`, `seller_state`)

Não foram identificados valores nulos.

Todas as siglas encontradas correspondem a unidades federativas válidas do Brasil.

In [7]:
nulos_customer = customers["customer_state"].isna().sum()
pct_customer = nulos_customer / len(customers) * 100

nulos_seller = sellers["seller_state"].isna().sum()
pct_seller = nulos_seller / len(sellers) * 100

print(f"customer_state - Nulos: {nulos_customer} ({pct_customer:.2f}%)")
print(f"seller_state   - Nulos: {nulos_seller} ({pct_seller:.2f}%)")

ufs_validas = {
    "AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS",
    "MG","PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC",
    "SP","SE","TO"
}

invalid_customer = customers.loc[
    ~customers["customer_state"].isin(ufs_validas),
    "customer_state"
].value_counts()

invalid_seller = sellers.loc[
    ~sellers["seller_state"].isin(ufs_validas),
    "seller_state"
].value_counts()

print("Siglas inválidas em customer_state:")
display(invalid_customer)

print("Siglas inválidas em seller_state:")
display(invalid_seller)

customer_state - Nulos: 0 (0.00%)
seller_state   - Nulos: 0 (0.00%)
Siglas inválidas em customer_state:


Series([], Name: count, dtype: int64)

Siglas inválidas em seller_state:


Series([], Name: count, dtype: int64)

## Geolocalização (`geolocation_lat`, `geolocation_lng`)

Foram identificados alguns registros com coordenadas geográficas fora dos limites territoriais do Brasil.

Esses registros foram **apenas identificados nesta etapa** e **serão tratados posteriormente** na etapa de limpeza dos dados.

Não foram identificados CEPs sem coordenadas geográficas válidas

Além disso, observou-se que o número total de registros é muito superior ao número de CEPs únicos, indicando que **um mesmo CEP possui múltiplas coordenadas**.

Para evitar isso, esses registros **serão agrupados pela média** das coordenadas na próxima etapa, definindo um ponto único de referência para cada CEP.

In [8]:
n = len(geolocation)

# Limites geográficos do Brasil convertido em graus decimais.
lat_norte = 5 + 16/60 + 19/3600 # Latitude Norte: 5°16'19" N
lat_sul = -(33 + 45/60 + 9/3600) # Latitude Sul: 33°45'09" S
lon_leste = -(34 + 45/60 + 54/3600) # Longitude Leste: 34°45'54" W
lon_oeste = -(73 + 59/60 + 32/3600) # Longitude Oeste: 73°59'32" W

fora_brasil = geolocation[
    (geolocation["geolocation_lat"] < lat_sul) |
    (geolocation["geolocation_lat"] > lat_norte) |
    (geolocation["geolocation_lng"] < lon_oeste) |
    (geolocation["geolocation_lng"] > lon_leste)
]

qtd_fora = len(fora_brasil)
pct_fora = qtd_fora / n * 100

print(f"Coordenadas fora do Brasil: {qtd_fora} ({pct_fora:.2f}%)")

nulos_geo = geolocation[
    geolocation[["geolocation_lat", "geolocation_lng"]].isna().any(axis=1)
]

qtd_nulos = len(nulos_geo)
pct_nulos = qtd_nulos / n * 100

print(f"CEPs sem coordenadas: {qtd_nulos} ({pct_nulos:.2f}%)")

ceps_unicos = geolocation['geolocation_zip_code_prefix'].nunique()
razao = n / ceps_unicos

print(f"Total de registros de geolocalização: {n}")
print(f"Total de CEPs únicos (prefixo): {ceps_unicos}")

Coordenadas fora do Brasil: 42 (0.00%)
CEPs sem coordenadas: 0 (0.00%)
Total de registros de geolocalização: 1000163
Total de CEPs únicos (prefixo): 19015


## Validação de Preço e Frete em Pedidos com Múltiplos Itens

Foi identificado que, em pedidos com múltiplos itens, os valores de `price` e `freight_value` frequentemente aparecem repetidos entre os itens. Isso levantou a dúvida se esses valores representariam o total do pedido ou custos unitários que deveriam ser somados.

Para validar essa questão, foram analisados apenas pedidos com mais de um item, comparando:

- O valor total calculado pela soma dos preços e fretes dos itens (`order_items`)
- O valor total efetivamente pago pelo cliente (`order_payments`)

Considerou-se como válido o caso em que a diferença entre os valores fosse inferior a R$ 0,10.

A análise contemplou **9.802 pedidos com múltiplos itens**, dos quais:

- **99,77%** apresentaram equivalência entre o valor calculado e o valor pago
- **0,23%** apresentaram pequenas divergências

Conclui-se que os valores de `price` e `freight_value` devem ser tratados como **custos unitários**, mesmo quando aparecem repetidos, e **devem ser somados** para o cálculo correto do valor total do pedido.


In [9]:
# Pedidos com mais de um item
ids_multi = (
    order_items
    .groupby("order_id")
    .size()
    .loc[lambda x: x > 1]
    .index
)

items_multi = order_items.query("order_id in @ids_multi")
payments_multi = payments.query("order_id in @ids_multi")

previsto = (
    items_multi
    .groupby("order_id", as_index=False)
    .agg(
        total_price=("price", "sum"),
        total_freight=("freight_value", "sum")
    )
)

previsto["total_previsto"] = previsto["total_price"] + previsto["total_freight"]

pago = (
    payments_multi
    .groupby("order_id", as_index=False)
    .agg(total_pago=("payment_value", "sum"))
)


prova = (
    previsto
    .merge(pago, on="order_id", how="inner")
    .assign(diferenca=lambda df: df["total_previsto"] - df["total_pago"])
)

acertos = prova[prova["diferenca"].abs() <= 0.1]
erros = prova[prova["diferenca"].abs() > 0.1]


total = len(prova)

print(f"Pedidos analisados: {total}")
print(f"Soma consistente com pagamento: {len(acertos)} ({len(acertos)/total:.2%})")
print(f"Divergências relevantes: {len(erros)} ({len(erros)/total:.2%})")

# Exemplo ilustrativo de pedido com múltiplos itens e valores repetidos
exemplo_id = acertos["order_id"].iloc[0]

print(f"Exemplo de pedido com múltiplos itens: {exemplo_id}")

display(
    order_items
    .loc[order_items["order_id"] == exemplo_id,
         ["order_item_id", "product_id", "price", "freight_value"]]
)

Pedidos analisados: 9802
Soma consistente com pagamento: 9779 (99.77%)
Divergências relevantes: 23 (0.23%)
Exemplo de pedido com múltiplos itens: 0008288aa423d2a3f00fcb17cd7d8719


,order_item_id,product_id,price,freight_value
13,1,368c6c730842d78016ad823897a372db,49.9,13.37
14,2,368c6c730842d78016ad823897a372db,49.9,13.37
